# Biological signals exploration

This notebook is about the exploration of the biosignals. We need urine level, creatine level (and evolution), as well as other stuff like dialysis.

In [17]:
import psycopg2
import pandas as pd

In [65]:
# create a database connection
sqluser = 'mimicuser'
sqlpassword = 'mimicpass'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, host='localhost', user=sqluser, password=sqlpassword)
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema_name))

In [42]:
query = \
"""
SELECT * FROM d_items WHERE label ~* '.*urine.*' AND linksto='outputevents';
"""

urine_items = pd.read_sql_query(query,con)
print(urine_items.row_id.count())
urine_items.head()

107


,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,5612,40405,Urine Out Other,None,carevue,outputevents,None,None,None,None
1,5629,40428,Urine Out Straight Cath,None,carevue,outputevents,None,None,None,None
2,5706,40534,urine flush,None,carevue,outputevents,Free Form Intake,None,None,None
3,6503,41857,urine out in er,None,carevue,outputevents,None,None,None,None
4,6607,42001,ER URINE,None,carevue,outputevents,None,None,None,None


In [28]:
query = \
"""
DROP MATERIALIZED VIEW IF EXISTS urineoutput CASCADE;
CREATE MATERIALIZED VIEW urineoutput as
select oe.icustay_id, oe.charttime
, SUM(
    -- we consider input of GU irrigant as a negative volume
    case when oe.itemid = 227488 then -1*value
    else value end
  ) as value
from outputevents oe
where oe.itemid in
(
  -- these are the most frequently occurring urine output observations in CareVue
  40055, -- "Urine Out Foley"
  43175, -- "Urine ."
  40069, -- "Urine Out Void"
  40094, -- "Urine Out Condom Cath"
  40715, -- "Urine Out Suprapubic"
  40473, -- "Urine Out IleoConduit"
  40085, -- "Urine Out Incontinent"
  40057, -- "Urine Out Rt Nephrostomy"
  40056, -- "Urine Out Lt Nephrostomy"
  40405, -- "Urine Out Other"
  40428, -- "Urine Out Straight Cath"
  40086,--	Urine Out Incontinent
  40096, -- "Urine Out Ureteral Stent #1"
  40651, -- "Urine Out Ureteral Stent #2"

  -- these are the most frequently occurring urine output observations in MetaVision
  226559, -- "Foley"
  226560, -- "Void"
  226561, -- "Condom Cath"
  226584, -- "Ileoconduit"
  226563, -- "Suprapubic"
  226564, -- "R Nephrostomy"
  226565, -- "L Nephrostomy"
  226567, --	Straight Cath
  226557, -- R Ureteral Stent
  226558, -- L Ureteral Stent
  227488, -- GU Irrigant Volume In
  227489  -- GU Irrigant/Urine Volume Out
)
and oe.value < 5000 -- sanity check on urine value
and oe.icustay_id is not null
group by icustay_id, charttime;

SELECT * FROM urineoutput;
"""

urine = pd.read_sql_query(query,con)
urine.head()

,icustay_id,charttime,value
0,200001,2181-11-25 22:02:00,250.0
1,200001,2181-11-26 20:47:00,60.0
2,200001,2181-11-28 09:35:00,50.0
3,200003,2199-08-02 20:15:00,230.0
4,200003,2199-08-02 22:00:00,0.0


In [39]:
urine[urine.icustay_id==200003].head(30)

,icustay_id,charttime,value
3,200003,2199-08-02 20:15:00,230.0
4,200003,2199-08-02 22:00:00,0.0
5,200003,2199-08-02 23:00:00,90.0
6,200003,2199-08-03 00:00:00,37.0
7,200003,2199-08-03 01:00:00,220.0
8,200003,2199-08-03 02:00:00,120.0
9,200003,2199-08-03 03:00:00,275.0
10,200003,2199-08-03 03:15:00,300.0
11,200003,2199-08-03 03:45:00,360.0
12,200003,2199-08-03 04:00:00,320.0


In [48]:
query=\
"""
SELECT * FROM d_items WHERE label ~* '.*creatinine.*';
"""
    
creatine_items =pd.read_sql_query(query,con)
print(creatine_items.row_id.count())
creatine_items.head(8)

8


,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,710,791,Creatinine (0-1.3),None,carevue,chartevents,Chemistry,None,None,None
1,1914,5811,urine creatinine,None,carevue,chartevents,None,None,None,None
2,2871,3750,Creatinine (0-0.7),None,carevue,chartevents,Chemistry,None,None,None
3,2977,1525,Creatinine,None,carevue,chartevents,Chemistry,None,None,None
4,14682,227005,Creatinine_ApacheIV,Creatinine_ApacheIV,metavision,chartevents,Scores - APACHE IV (2),None,Numeric,None
5,12698,220615,Creatinine,Creatinine,metavision,chartevents,Labs,None,Numeric,None
6,14512,226751,CreatinineApacheIIScore,CreatinineApacheIIScore,metavision,chartevents,Scores - APACHE II,None,Numeric,None
7,14513,226752,CreatinineApacheIIValue,CreatinineApacheIIValue,metavision,chartevents,Scores - APACHE II,None,Numeric,None


In [54]:
query=\
"""
SELECT * FROM chartevents WHERE itemid=1525 AND subject_id=21 LIMIT 30;
"""

creatinine = pd.read_sql_query(query, con)
creatinine.head(100)

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,34350768,21,111970,216859,1525,2135-02-04 04:08:00,2135-02-04 05:40:00,15331,2.4,2.4,None,None,None,Final,NotStopd
1,34352355,21,111970,216859,1525,2135-02-05 22:54:00,2135-02-05 23:37:00,15331,2,2.0,None,None,None,Final,NotStopd
2,34343799,21,109451,217847,1525,2134-09-13 02:58:00,2134-09-13 04:38:00,15331,3.2,3.2,None,None,None,Final,NotStopd
3,34348323,21,111970,216859,1525,2135-02-01 02:59:00,2135-02-01 03:51:00,15331,4.7,4.7,None,None,None,Final,NotStopd
4,34344876,21,109451,217847,1525,2134-09-13 21:37:00,2134-09-13 22:54:00,15331,2.5,2.5,None,None,None,Final,NotStopd
5,34347022,21,109451,217847,1525,2134-09-17 03:14:00,2134-09-17 04:38:00,15331,3.8,3.8,None,None,None,Final,NotStopd
6,34346105,21,109451,217847,1525,2134-09-15 14:10:00,2134-09-15 15:09:00,15331,3.6,3.6,None,None,None,Final,NotStopd
7,34349209,21,111970,216859,1525,2135-02-02 05:03:00,2135-02-02 05:50:00,15331,2.5,2.5,None,None,None,Final,NotStopd
8,34342806,21,109451,217847,1525,2134-09-12 04:30:00,2134-09-12 05:31:00,15331,4,4.0,None,None,None,Final,NotStopd
9,34345434,21,109451,217847,1525,2134-09-14 10:30:00,2134-09-14 11:25:00,15331,2.4,2.4,None,None,None,Final,NotStopd


In [69]:
query=\
"""
SELECT * FROM d_items WHERE label ~* '.*dialysis.*';
"""
    
dialysis_items = pd.read_sql_query(query,con)
print(dialysis_items.row_id.count())
dialysis_items.head(100)

99


,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,149,148,Dialysis Access Site,None,carevue,chartevents,None,None,None,None
1,150,149,Dialysis Access Type,None,carevue,chartevents,None,None,None,None
2,151,150,Dialysis Machine,None,carevue,chartevents,None,None,None,None
3,152,151,Dialysis Site Appear,None,carevue,chartevents,None,None,None,None
4,153,152,Dialysis Type,None,carevue,chartevents,None,None,None,None
5,5628,40425,dialysis output,None,carevue,outputevents,None,None,None,None
6,5681,40507,Dialysis out,None,carevue,outputevents,None,None,None,None
7,6122,41527,HEMODIALYSIS,None,carevue,outputevents,None,None,None,None
8,6964,41250,HEMODIALYSIS OUT,None,carevue,outputevents,None,None,None,None
9,7065,41374,Dialysis Out,None,carevue,outputevents,None,None,None,None


In [68]:
query=\
"""
SELECT * FROM chartevents WHERE itemid=152;
"""

dialysis = pd.read_sql_query(query, con)
dialysis.value.value_counts()

CVVHD         50805
CVVH           8777
Peritoneal      206
Name: value, dtype: int64